**Занятие первое**

Начнем с простого. Многие знают что такое map и reduce операции, но все же для закрпеления мы их тут реализуем. Ах да, не забудем и про shuffle. Делать все будем на упрощенной задаче с word count для ознакомления с самим подходом.

На самом деле мы рассмптрим все в упрощенном виде, но это даст нам понимание, как можно через hadoop streaming, например, писать самописные map и reduce операции

! mapred streaming \
  -input /wiki/sample.jsonl \
  -output /word-count \
  -mapper "/opt/conda/bin/python3.6 mapper.py" \
  -reducer "/opt/conda/bin/python3.6 reducer.py" \
  -file mapper.py \
  -file reducer.py

Выше mapper.py и reducer.py это программы, которые выполняют одноименные операции нам потоком информации из jsonl файла, записывая ответ в файл word-count

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re 
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Давайте загрузим файл с текстом и посмотрим на него

In [ ]:
with open('spark_text.txt', 'rb') as f:
    data = f.readlines()
data = [text.decode() for text in data if text.decode() != '\r\n']    

In [ ]:
len(data)

100

In [ ]:
data[1]

'\n'

Как бы мы сделали..
Надо немного почистить слова, а также сделать все в парадигме MapReduce. Понятно, что можно все написать проще, но мы ведь хотим понять, как это работает=)

Загрузим стоп слова, очистим от них текст, приведем к нижнему регистру, всем раздадим ключи

In [ ]:
stop_words = stopwords.words("english")
stop_words = set(stop_words)

In [ ]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

пунктуацию тоже полезно бы удалить

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def mapper_text(text):
    clean_text = re.sub(rf"[{string.punctuation}]", "", text)
    words = nltk.word_tokenize(clean_text)
    words_with_value = [(word.lower(), 1) for word in words 
                        if word not in stop_words]
    words_with_value = sorted(words_with_value, key=lambda x:x[0])
    return words_with_value

def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())

def shuffle_text(mapper_result, n_nodes=5):
    shuffled_data = []
    for key, value in mapper_result:
        shuffled_data.append((hash(key)%n_nodes, (key, value)))
    shuffled_data = sorted(shuffled_data, key=lambda x: x[0])
    chunks = create_chunks(shuffled_data)
    return chunks


# на самом деле для reduce в жизни пишут иначе..не зря мы сортируем внутри map
#данные по ключам. Это нужно для избавления от этапа проверки ключа и поиска
def reduce_text(values_to_reduce):
    result = {}
    for key, value in values_to_reduce:
        if key in result:
            result[key] += 1
        else:
            result[key] = 1
    return result

Проверим, что все работает

Сначала map

In [ ]:
data[0]

'Apache Spark is an open-source unified analytics engine for large-scale data processing. Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance. Originally developed at the University of California, Berkeley AMPLab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since.\n'

In [ ]:
map_stage = mapper_text(data[0])

In [ ]:
map_stage

[('amplab', 1),
 ('analytics', 1),
 ('apache', 1),
 ('apache', 1),
 ('berkeley', 1),
 ('california', 1),
 ('clusters', 1),
 ('codebase', 1),
 ('data', 1),
 ('data', 1),
 ('developed', 1),
 ('donated', 1),
 ('engine', 1),
 ('entire', 1),
 ('fault', 1),
 ('foundation', 1),
 ('implicit', 1),
 ('interface', 1),
 ('largescale', 1),
 ('later', 1),
 ('maintained', 1),
 ('opensource', 1),
 ('originally', 1),
 ('parallelism', 1),
 ('processing', 1),
 ('programming', 1),
 ('provides', 1),
 ('since', 1),
 ('software', 1),
 ('spark', 1),
 ('spark', 1),
 ('spark', 1),
 ('tolerance', 1),
 ('unified', 1),
 ('university', 1)]

shuffle

In [ ]:
shuffle_stage = shuffle_text(map_stage, 5)

In [ ]:
shuffle_stage

[(0,
  [('berkeley', 1),
   ('clusters', 1),
   ('developed', 1),
   ('foundation', 1),
   ('implicit', 1),
   ('interface', 1),
   ('later', 1),
   ('parallelism', 1),
   ('unified', 1)]),
 (1,
  [('amplab', 1),
   ('analytics', 1),
   ('california', 1),
   ('data', 1),
   ('data', 1),
   ('engine', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1),
   ('university', 1)]),
 (2, [('apache', 1), ('apache', 1), ('donated', 1), ('processing', 1)]),
 (3,
  [('entire', 1),
   ('fault', 1),
   ('largescale', 1),
   ('opensource', 1),
   ('programming', 1),
   ('provides', 1),
   ('software', 1)]),
 (4,
  [('codebase', 1),
   ('maintained', 1),
   ('originally', 1),
   ('since', 1),
   ('tolerance', 1)])]

reduce

In [ ]:
reduce_text(shuffle_stage[1][1])

{'amplab': 1,
 'analytics': 1,
 'california': 1,
 'data': 2,
 'engine': 1,
 'spark': 3,
 'university': 1}

Итак, осталось все рассчитать параллельно и собрать результаты

In [2]:
from joblib import Parallel, delayed

In [ ]:
n_nodes = 5

Обернем в 1 функциию для удобства map и shuffle

In [ ]:
def map_shuffle(text, n_nodes):
    map_result = mapper_text(text)
    shuffle_result = shuffle_text(map_result, n_nodes)
    return shuffle_result

In [ ]:
[delayed(map_shuffle)(df, n_nodes) for df in data]

[(<function __main__.map_shuffle(text, n_nodes)>,
  ('Apache Spark is an open-source unified analytics engine for large-scale data processing. Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance. Originally developed at the University of California, Berkeley AMPLab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since.\n',
   5),
  {}),
 (<function __main__.map_shuffle(text, n_nodes)>, ('\n', 5), {}),
 (<function __main__.map_shuffle(text, n_nodes)>,
  ('Apache Spark has its architectural foundation in the resilient distributed dataset (RDD), a read-only multiset of data items distributed over a cluster of machines, that is maintained in a fault-tolerant way.[2] The Dataframe API was released as an abstraction on top of the RDD, followed by the Dataset API. In Spark 1.x, the RDD was the primary application programming interface (API), but as of Spark 2.x use of the Dataset API is

In [ ]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(map_shuffle)(df, n_nodes) for df in data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    2.2s
[Parallel(n_jobs=5)]: Done  25 tasks      | elapsed:    2.3s
[Parallel(n_jobs=5)]: Done  50 tasks      | elapsed:    2.3s
[Parallel(n_jobs=5)]: Done  85 tasks      | elapsed:    2.3s
[Parallel(n_jobs=5)]: Done  92 tasks      | elapsed:    2.3s
[Parallel(n_jobs=5)]: Done 100 out of 100 | elapsed:    2.3s finished


In [ ]:
len(res)

100

In [ ]:
res[0]

[(0,
  [('analytics', 1),
   ('california', 1),
   ('donated', 1),
   ('implicit', 1),
   ('interface', 1),
   ('originally', 1)]),
 (1,
  [('software', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1),
   ('university', 1)]),
 (2, [('berkeley', 1), ('maintained', 1), ('unified', 1)]),
 (3,
  [('amplab', 1),
   ('apache', 1),
   ('apache', 1),
   ('data', 1),
   ('data', 1),
   ('developed', 1),
   ('engine', 1),
   ('programming', 1)]),
 (4,
  [('clusters', 1),
   ('codebase', 1),
   ('entire', 1),
   ('fault', 1),
   ('foundation', 1),
   ('largescale', 1),
   ('later', 1),
   ('opensource', 1),
   ('parallelism', 1),
   ('processing', 1),
   ('provides', 1),
   ('since', 1),
   ('tolerance', 1)])]

Сделаем что-то вроде перессылки, собирая все в словари и заодно посмотрим на сколько равномерно распределлиись наши слова

In [ ]:
shuffle_stage = {i:[] for i in range(5)}
for values in res:
    values = dict(values)
    for key in values.keys():
        shuffle_stage[key].extend(values[key])

In [ ]:
for key in shuffle_stage.keys():
    print(f'{key}: number of words = {len(shuffle_stage[key])}')

0: number of words = 368
1: number of words = 349
2: number of words = 384
3: number of words = 405
4: number of words = 415


И последний этап - нужно сделать reduce

In [ ]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(reduce_text)(shuffle_stage[key]) for key in shuffle_stage.keys())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


In [ ]:
len(res)

5

In [ ]:
res[0]

{'analytics': 1,
 'california': 1,
 'donated': 1,
 'implicit': 1,
 'interface': 5,
 'originally': 2,
 'api': 8,
 'deprecated45': 1,
 'encouraged3': 1,
 'maintained': 1,
 'programming': 8,
 'spark': 34,
 'still': 2,
 'underlies': 1,
 'way2': 1,
 'cluster': 1,
 'read': 1,
 'reduce': 1,
 'results': 2,
 'store': 1,
 'memory8': 1,
 'offers': 3,
 'working': 1,
 'analysis': 1,
 'visit': 1,
 'among': 1,
 'apache': 12,
 'developing': 1,
 'formed': 1,
 'training': 1,
 'run': 1,
 'scripts': 1,
 'testing': 1,
 'alluxio': 1,
 'hadoop': 6,
 'local': 2,
 'machine': 3,
 'mode': 2,
 'openstack': 1,
 'supports': 3,
 'swift': 1,
 'centered': 1,
 'dispatching': 1,
 'exposed': 1,
 'jvm': 2,
 'overall': 1,
 'task': 1,
 'case': 1,
 'contain': 1,
 'filter': 1,
 'immutable': 2,
 'new': 1,
 'objects': 2,
 'rdds': 7,
 'sequence': 2,
 'this': 2,
 'accumulators': 1,
 'rddoriented': 1,
 'readonly': 1,
 'reference': 1,
 'shared': 1,
 'variables': 2,
 'each': 1,
 'following': 1,
 'frequencies': 1,
 'takes': 2,
 'vari

Собираем результат

In [ ]:
result = {}
for partition in res:
    for key in partition.keys():
        if key in result:
            result[key] += partition[key]
        else:
            result[key] = partition[key]

In [ ]:
sorted(result.items(), key=lambda x: x[1], reverse=True)

[('spark', 110),
 ('apache', 56),
 ('data', 44),
 ('streaming', 36),
 ('distributed', 23),
 ('processing', 19),
 ('sql', 17),
 ('rdd', 15),
 ('learning', 15),
 ('also', 15),
 ('api', 14),
 ('structured', 13),
 ('cluster', 12),
 ('machine', 12),
 ('hadoop', 11),
 ('interface', 10),
 ('rdds', 10),
 ('mapreduce', 10),
 ('provides', 10),
 ('programming', 9),
 ('mllib', 9),
 ('graph', 9),
 ('framework', 9),
 ('the', 9),
 ('use', 9),
 ('core', 9),
 ('used', 8),
 ('support', 8),
 ('application', 8),
 ('python', 8),
 ('code', 8),
 ('tasks', 8),
 ('run', 7),
 ('graphx', 7),
 ('algorithms', 7),
 ('operations', 7),
 ('deep', 7),
 ('it', 7),
 ('dataset', 7),
 ('in', 7),
 ('abstraction', 6),
 ('batch', 6),
 ('graphs', 6),
 ('pipelines', 6),
 ('including', 6),
 ('two', 6),
 ('developers', 6),
 ('big', 6),
 ('scala', 6),
 ('map', 6),
 ('simple', 6),
 ('implementation', 6),
 ('analytics', 5),
 ('supports', 5),
 ('new', 5),
 ('writing', 5),
 ('applications', 5),
 ('set', 5),
 ('even', 5),
 ('top', 5),


Да, было бы проще все сделать иным кодом и в один проход, но целью было разобрать, как все это примерно работает под капотом на больших данных.

**Домашнее задание**

Посчитать количество рейтингов больше 4 для каждого фильма и вывести фильмы в порядке убывания количества этих оценок

In [9]:
with open('user_ratedmovies.dat', 'rb') as f:
    data = f.readlines()
headers = data[0].decode().split('\t')[:3]
data = [row.decode().split('\t')[:3] for row in data[1:]]

In [10]:
headers

['userID', 'movieID', 'rating']

In [11]:
data[0]

['75', '3', '1']

In [12]:
len(data)

855598

Пишем map, shiffle и reduce + параллелим вычисления. Лучше задавать batch_size при распараллеливании, либо даже заранее все разбить на батчи, будет быстрее

Также посмотрите на то, нет ли перекоса в данных после shuffle, можете попробовать использовать остаток от деления не простого hash, а ввести какую-то функию

* Представлю, что у меня 20 Data Node и все работают, тогда поделю всю выборку на 20 батчей, каждый из которых будет храниться и мапиться на своей ноде

In [126]:
def map_rating(film_with_rating):

    film_with_rating = (int(film_with_rating[1]), (1 if float(film_with_rating[2])>4 else 0))
    return film_with_rating

def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())

def shuffle_rating(mapper_result, n_nodes=5):
    # shuffled_data = []
    # for key, value in mapper_result:
    #     shuffled_data.append((hash(key)%n_nodes, (key, value)))
    # shuffled_data = sorted(shuffled_data, key=lambda x: x[0])
    # chunks = create_chunks(shuffled_data)
    return (hash(mapper_result[0])%n_nodes, (mapper_result[0], mapper_result[1]))

def reduce_rating(values_to_reduce):
    result = {}
    for key, value in values_to_reduce:
        if key in result:
            result[key] += value
        else:
            result[key] = value
    return result

def map_shuffle(data, n_nodes):
    map_stage = map_rating(data)
    shuffle_stage = shuffle_rating(map_stage, n_nodes)
    return shuffle_stage

In [124]:
data[0], map_rating(data[0])

(['75', '3', '1'], (3, 0))

In [127]:
shuffle_rating(map_rating(data[0])), shuffle_rating(map_rating(data[100]))

((3, (3, 0)), (4, (904, 1)))

In [128]:
map_shuffle(data[0], 5)

(3, (3, 0))

In [129]:
with Parallel(n_jobs=20, verbose=-1, batch_size=1) as parallel:
    res = parallel(delayed(map_shuffle)(df, 5) for df in data)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  37 tasks      | elapsed:    0.3s
[Parallel(n_jobs=20)]: Done 253 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 613 tasks      | elapsed:    2.2s
[Parallel(n_jobs=20)]: Done 1117 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 1765 tasks      | elapsed:    3.6s
[Parallel(n_jobs=20)]: Done 2557 tasks      | elapsed:    4.4s
[Parallel(n_jobs=20)]: Done 3493 tasks      | elapsed:    5.5s
[Parallel(n_jobs=20)]: Done 4573 tasks      | elapsed:    6.4s
[Parallel(n_jobs=20)]: Done 5797 tasks      | elapsed:    7.3s
[Parallel(n_jobs=20)]: Done 7165 tasks      | elapsed:    8.3s
[Parallel(n_jobs=20)]: Done 8677 tasks      | elapsed:    9.4s
[Parallel(n_jobs=20)]: Done 10333 tasks      | elapsed:   10.6s
[Parallel(n_jobs=20)]: Done 12133 tasks      | elapsed:   12.0s
[Parallel(n_jobs=20)]: Done 14077 tasks      | elapsed:   13.4s
[Parallel(n_jobs=20)]: Done 16165 tasks  

In [130]:
res[:10]

[(3, (3, 0)),
 (2, (32, 1)),
 (0, (110, 0)),
 (0, (160, 0)),
 (3, (163, 0)),
 (0, (165, 1)),
 (3, (173, 0)),
 (1, (296, 1)),
 (3, (353, 0)),
 (0, (420, 0))]

In [131]:
#перегоняю все на 5 нод для reduce на каждой
shuffle_stage = {i:[] for i in range(5)}
for key, value in res:
    shuffle_stage[key].append(value)

In [132]:
# данные после мапа разлетелись на 5 нод для reduce
len(shuffle_stage)

5

In [133]:
shuffle_stage[0][:10]

[(110, 0),
 (160, 0),
 (165, 1),
 (420, 0),
 (920, 0),
 (1215, 1),
 (1370, 0),
 (1485, 0),
 (2490, 0),
 (2640, 0)]

In [134]:
for key in shuffle_stage.keys():
    print(f'{key}: number of rows = {len(shuffle_stage[key])}')

0: number of rows = 170211
1: number of rows = 169817
2: number of rows = 170990
3: number of rows = 183481
4: number of rows = 161099


In [135]:
with Parallel(n_jobs=5, verbose=10, batch_size=1) as parallel:
    res = parallel(delayed(reduce_rating)(shuffle_stage[key]) for key in shuffle_stage.keys())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.4s remaining:    0.6s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.5s remaining:    0.3s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.7s finished


In [136]:
len(res)

5

In [141]:
# Соедиеняем все выходы reduce для итогового результата
total_res = {}

for r in res:
    total_res.update(r)

In [145]:
sorted(total_res.items(), key=lambda item: item[1], reverse=True)[:10]

[(2571, 900),
 (318, 887),
 (296, 878),
 (2959, 828),
 (4993, 756),
 (7153, 719),
 (5952, 697),
 (858, 690),
 (50, 688),
 (2858, 680)]

In [139]:
import pandas as pd
import numpy as np
pd.DataFrame(columns=headers, data=np.array(data).astype(np.float)).query("rating>4")\
                                    .groupby('movieID').count()\
                                    .sort_values(by='rating', ascending=False)\
                                    .head(10)

<ipython-input-139-8a6dd900f7cc>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pd.DataFrame(columns=headers, data=np.array(data).astype(np.float)).query("rating>4")\


,userID,rating
movieID,,
2571.0,900,900
318.0,887,887
296.0,878,878
2959.0,828,828
4993.0,756,756
7153.0,719,719
5952.0,697,697
858.0,690,690
50.0,688,688
